In [1]:
import pandas as pd

In [2]:
aircraft_models = pd.read_csv('aircraft_model_info.csv')
aircraft_models = aircraft_models.drop(columns=['Unnamed: 13', 'TC-DATA-HOLDER', 'TC-DATA-SHEET', 'AC-CAT', 'BUILD-CERT-IND', 'TYPE-ACFT', 'SPEED'])
aircraft_models = aircraft_models[aircraft_models['AC-WEIGHT'].str.contains('CLASS 3')]
aircraft_models = aircraft_models[aircraft_models['NO-SEATS'] > 100]
aircraft_models.head()

,CODE,MFR,MODEL,TYPE-ENG,NO-ENG,NO-SEATS,AC-WEIGHT
36107,05628NI,BOEING,787-8,5,2,260,CLASS 3
37699,05629VS,BOEING,787-83Q,5,2,260,CLASS 3
47068,056374G,EMBRAER,190-100 AR,5,2,114,CLASS 3
47218,056378M,BOEING,777-9,5,2,130,CLASS 3
75890,138004A,BOEING,777-28EER,5,2,270,CLASS 3


In [3]:
aircraft_models.__len__()

1680

In [15]:
aircraft_registration = pd.read_csv('aircraft_registration_info.csv')
aircraft_info = pd.merge(aircraft_models, aircraft_registration, left_on='CODE', right_on='MFR MDL CODE',how='inner')

/var/folders/85/81_gjq057hg3bss019j8x98r0000gp/T/ipykernel_23615/4093725381.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  aircraft_registration = pd.read_csv('aircraft_registration_info.csv')


In [16]:
aircraft_info = aircraft_info.drop(columns=['ENG MFR MDL','STREET','STREET2','CITY','STATE','ZIP CODE','REGION','COUNTY','LAST ACTION DATE','CERT ISSUE DATE','CERTIFICATION','TYPE AIRCRAFT','TYPE ENGINE','STATUS CODE','MODE S CODE','FRACT OWNER','AIR WORTH DATE','OTHER NAMES(1)','OTHER NAMES(2)','OTHER NAMES(3)','OTHER NAMES(4)','OTHER NAMES(5)','EXPIRATION DATE','UNIQUE ID','KIT MFR', ' KIT MODEL','Unnamed: 34','MODE S CODE HEX', 'COUNTRY', 'TYPE REGISTRANT', 'MFR MDL CODE', 'SERIAL NUMBER', 'AC-WEIGHT'])
aircraft_info.head()

,CODE,MFR,MODEL,TYPE-ENG,NO-ENG,NO-SEATS,N-NUMBER,YEAR MFR,NAME
0,05628NI,BOEING,787-8,5,2,260,7874,2010,BOEING CO ...
1,05628NI,BOEING,787-8,5,2,260,787BX,2010,BOEING CO ...
2,05628NI,BOEING,787-8,5,2,260,791AV,2016,BANK OF UTAH TRUSTEE ...
3,05628NI,BOEING,787-8,5,2,260,877BF,2022,WILMINGTON TRUST CO TRUSTEE ...
4,05628NI,BOEING,787-8,5,2,260,884AA,2022,WILMINGTON TRUST CO TRUSTEE ...


In [17]:
# Export to CSV
aircraft_info.to_csv('aircraft_info.csv', index=False)

In [18]:
# remove duplicate 'N-NUMBER' rows
print(aircraft_info.__len__())
aircraft_info = aircraft_info.drop_duplicates(subset='N-NUMBER', keep='first')
print(aircraft_info.__len__())

6694
6694


In [19]:
airlines = pd.read_csv('airlines.csv')
airlines

,IATA,Airline,Country_Region
0,AA,American Airlines,United States
1,AC,Air Canada,Canada
2,AD,Azul Linhas Aéreas Brasileiras,Brazil
3,AE,Mandarin Airlines,Taiwan
4,AF,Air France,France
...,...,...,...
567,ZS,Sama Airlines,Saudi Arabia
568,ZT,Titan Airways,United Kingdom
569,ZW,Air Wisconsin,United States
570,ZX,Air Georgian,Canada


In [22]:
import random

# Function to get a random airline based on country
def get_random_airline(country):
    if random.random() < 0.5:
        return random.choice(['DL', 'AA', 'WN', 'UA'])
    matching_airlines = airlines[airlines['Country_Region'] == country]
    if not matching_airlines.empty:
        return random.choice(matching_airlines['IATA'].values)
    else:
        return random.choice(airlines['IATA'].values)

chosen_country = 'United States'
# Apply the function to each aircraft
aircraft_info['airline_id'] = aircraft_info.apply(lambda row: get_random_airline(chosen_country), axis=1)

aircraft_info.head()

,CODE,MFR,MODEL,TYPE-ENG,NO-ENG,NO-SEATS,N-NUMBER,YEAR MFR,NAME,airline_id
0,05628NI,BOEING,787-8,5,2,260,7874,2010,BOEING CO ...,DL
1,05628NI,BOEING,787-8,5,2,260,787BX,2010,BOEING CO ...,UA
2,05628NI,BOEING,787-8,5,2,260,791AV,2016,BANK OF UTAH TRUSTEE ...,QX
3,05628NI,BOEING,787-8,5,2,260,877BF,2022,WILMINGTON TRUST CO TRUSTEE ...,WN
4,05628NI,BOEING,787-8,5,2,260,884AA,2022,WILMINGTON TRUST CO TRUSTEE ...,WN


In [27]:
def castInt(x):
    try:
        return int(x)
    except:
        return 0

In [29]:
import numpy as np
# Format each row for SQL entry
formatted_rows = []
for index, row in aircraft_info.iterrows():
    tail_no = row['N-NUMBER']
    model = row['MFR'].strip() + ' ' + row['MODEL'].strip()
    manufacturing_year = np.max([castInt(row['YEAR MFR']), 1989])
    capacity = row['NO-SEATS']
    airline_id = row['airline_id']
    formatted_row = f"INSERT INTO aircraft VALUES('{tail_no}', '{model}', {manufacturing_year}, {capacity}, '{airline_id}')"
    formatted_rows.append(formatted_row)

# Join all formatted rows into a single string
formatted_text = ';\n'.join(formatted_rows)

# Export the formatted text to a plaintext file
with open('aircraft_code_sql_entries.txt', 'w') as file:
    file.write(formatted_text)